# TD Models with intermixing Time scales

In class we discussed that TD methods can also be used to learn a model of the environment. These models can be used with model based RL architectures. Model-based reinforcement learning allows us to elegantly integrate planning into a real-time learning and decision-making agent.DYNA is one of the preferred architectures for this.

Planning can be made much faster using models predicting further into the future - this allows the agent to look farther and get estimates with reduced bias and hence faster convergence. (Side note - I was working to show this very fact but increased variance in the n-Step model was not helping me. I was getting best results with n=1 :-( I'll try demonstrating this again for the next assignment with a few more environments)

n-Step models, therefore, can be used for lookahead to find state values - V just like the base model, while requiring significantly fewer steps. One step of lookahead with a 10-step model should exactly be equivalent to 10 steps of lookahead with a 1-step model.

However, n-Step models are limited in an important way- they insist on a single, fixed time interval n. Besides, it is computationally difficult to precisely learn these n-Step models. Such models are hence impractical and unnecessary. A preferred alternative would be to have temporally abstract prediction model which gives a weighted approximations of future states. [TD Models: Modeling the World at a Mixture of Time Scales](https://webdocs.cs.ualberta.ca/~sutton/papers/sutton-95.ps.gz) introduced the concept of intermixing different time scales for RL predicition problem. 

The paper suggests that a desired multi-scale model is a mixture of models for all different time scales - 
$$ P = \sum_{n=1}^{\infty} w_{n=1}P^{(n)}$$
$$ R = \sum_{n=1}^{\infty} w_{n=1}R^{(n)}$$
i.e. weighted average of simple 'i'-step model.

## $$\beta-Models$$

A preferred mixture of n-step models is in which weights fall of exponentially each time step into future. This comes from the fact that number of different future events possible from a point are exponential and hence their effects should have less and less importance.

$$ w_{n} = (1-\beta) \beta^{n-1} $$
for a fixed parameter between 0 and 1.

The paper introduced full-beta models. In these models, beta are allowed to vary from state to state and therefore trajectories have arbitrary weight vectors. 
$$ P = \gamma (I-B) P \sum_{k=0}^{\infty} (\gamma B P)^{k} $$
and 
$$ R = \sum_{k=0}^{\infty} (\gamma P^T B)^{k} R $$

## Wall Following domain

In the following section, I illustrate the effectiveness of these full-beta models in Wall Following domain. Wall-following is not a single action or sequence of actions, but a complete closed-loop policy for moving forward while staying close to a wall on one side. 

See the grid state bellow in the code section. 

The robot starts at one of the three cells at the bottom and travels up one row per time step. The stocastic policy tries to keep the robot in center column to avoid going into open spacetes.s or colliding with the wall. An episode under this task ends when robot either enters the 'Exit' section of grid, or collides with the 'Wall' or goes out into 'Open Space' and loses sensory contact.

In the following experiment I recreate the auther's attempt to learn a temporally abstact predictive model for likely future states. By setting beta of all grid cells at 1 and only giving terminal states beta value of 0, all the learning is about the final terminal states. Predictions for termination to all grid cells are 0 and only values learnt are for the states with beta=0. 

The expiremental probabilities of termination into Wall, Exit and Open Spaces match the expected correct predictions as in the paper as well.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
#All possible actions defined
ACTION_UP_LEFT = 'LEFT
ACTION_UP = 'UP'
ACTION_UP_RIGHT = 'RIGHT'

#All states in Wall-Following Grid
grid_states = [           'Exit',
                     '00', '01', '02',
                     '03', '04', '05',
                     '06', '07', '08',
                     '09', '10', '11',
              'Wall','12', '13', '14','Open Space',
                     '15', '16', '17',
                     '18', '19', '20',
                     '21', '22', '23']

# All terminal states
terminals = ['Exit', 'Wall', 'Open Space']

#All transitions in Dyna-Maze
all_transitions =  {
    '00': {ACTION_UP_LEFT : 'Wall', ACTION_UP : 'Exit', ACTION_UP_RIGHT : 'Exit'},
    '01': {ACTION_UP_LEFT : 'Exit', ACTION_UP : 'Exit', ACTION_UP_RIGHT : 'Exit'},
    '02': {ACTION_UP_LEFT : 'Exit', ACTION_UP : 'Exit', ACTION_UP_RIGHT : 'Open Space'},
    '03': {ACTION_UP_LEFT : 'Wall', ACTION_UP : '00',   ACTION_UP_RIGHT : '01'},
    '04': {ACTION_UP_LEFT : '00',   ACTION_UP : '01',   ACTION_UP_RIGHT : '02'},
    '05': {ACTION_UP_LEFT : '01',   ACTION_UP : '02',   ACTION_UP_RIGHT : 'Open Space'},
    '06': {ACTION_UP_LEFT : 'Wall', ACTION_UP : '03',   ACTION_UP_RIGHT : '04'},
    '07': {ACTION_UP_LEFT : '03',   ACTION_UP : '04',   ACTION_UP_RIGHT : '05'},
    '08': {ACTION_UP_LEFT : '04',   ACTION_UP : '05',   ACTION_UP_RIGHT : 'Open Space'},
    '09': {ACTION_UP_LEFT : 'Wall', ACTION_UP : '06',   ACTION_UP_RIGHT : '07'},
    '10': {ACTION_UP_LEFT : '06',   ACTION_UP : '07',   ACTION_UP_RIGHT : '08'},
    '11': {ACTION_UP_LEFT : '07',   ACTION_UP : '08',   ACTION_UP_RIGHT : 'Open Space'},
    '12': {ACTION_UP_LEFT : 'Wall', ACTION_UP: '09',    ACTION_UP_RIGHT : '10'},
    '13': {ACTION_UP_LEFT : '09',   ACTION_UP : '10',   ACTION_UP_RIGHT : '11'},
    '14': {ACTION_UP_LEFT : '10',   ACTION_UP : '11',   ACTION_UP_RIGHT : 'Open Space'},
    '15': {ACTION_UP_LEFT : 'Wall', ACTION_UP : '12',   ACTION_UP_RIGHT : '13'},
    '16': {ACTION_UP_LEFT : '12',   ACTION_UP : '13',   ACTION_UP_RIGHT : '14'},
    '17': {ACTION_UP_LEFT : '13',   ACTION_UP : '14',   ACTION_UP_RIGHT : 'Open Space'},
    '18': {ACTION_UP_LEFT : 'Wall', ACTION_UP : '15',   ACTION_UP_RIGHT : '16'},
    '19': {ACTION_UP_LEFT : '15',   ACTION_UP : '16',   ACTION_UP_RIGHT : '17'},
    '20': {ACTION_UP_LEFT : '16',   ACTION_UP : '17',   ACTION_UP_RIGHT : 'Open Space'},
    '21': {ACTION_UP_LEFT : 'Wall', ACTION_UP : '18',   ACTION_UP_RIGHT : '19'},
    '22': {ACTION_UP_LEFT : '18',   ACTION_UP : '19',   ACTION_UP_RIGHT : '20'},
    '23': {ACTION_UP_LEFT : '19',   ACTION_UP : '20',   ACTION_UP_RIGHT : 'Open Space'},
}

# Policy
policy = {
    '00': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '01': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '02': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6},
    '03': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '04': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '05': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6},
    '06': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '07': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '08': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6},
    '09': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '10': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '11': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6},
    '12': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '13': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '14': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6},
    '15': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '16': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '17': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6},
    '18': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '19': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '20': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6},
    '21': {ACTION_UP_LEFT : 1./6, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./2},
    '22': {ACTION_UP_LEFT : 1./4, ACTION_UP : 1./2, ACTION_UP_RIGHT : 1./4},
    '23': {ACTION_UP_LEFT : 1./2, ACTION_UP : 1./3, ACTION_UP_RIGHT : 1./6}
}

# beta model
beta = {
    'Exit': 0,
    '00' : 1,
    '01' : 1,
    '02' : 1,
    '03' : 1,
    '04' : 1,
    '05' : 1,
    '06' : 1,
    '07' : 1,
    '08' : 1,
    '09' : 1,
    '10' : 1,
    '11' : 1,
    'Wall' : 0,
    '12' : 1,
    '13' : 1,
    '14' : 1,
    'Open Space' : 0,
    '15' : 1,
    '16' : 1,
    '17' : 1,
    '18' : 1,
    '19' : 1,
    '20' : 1,
    '21' : 1,
    '22' : 1,
    '23' : 1
}

In [3]:
nTrails = 5000
GAMMA = 0.9
ALPHA = 0.01
trajectories=[]

p = np.zeros((len(grid_states),len(grid_states)))

def getFeature(state):
    f = np.zeros(len(grid_states))
    f[grid_states.index(state)]=1
    return f

for ep in range(nTrails):
    episode = []
    s = np.random.choice(['21','22','23'])
    episode.append(s)
    while s not in terminals:
        a = np.random.choice(policy[s].keys(), p=policy[s].values())
        next_s = all_transitions[s][a]
        episode.append(next_s)
        s= next_s
    trajectories.append(episode)
    
for t in trajectories:
    #print t
    for i,s in enumerate(t):
        f = getFeature(s)
        y = np.zeros(f.shape)
        beta_prod=1
        n = len(t[i+1:])
        for k in range(1,n+1):
            beta_prod *= beta[ t[i+k-1] ] if k>1 else 1
            y += GAMMA**k * (1-beta[ t[i+k] ]) * beta_prod * getFeature(t[i+k])
            #print s, k, t[i+k-1], t[i+k], y
        beta_prod *= beta[ t[i+n] ]
        y += GAMMA**n * beta_prod * np.dot(p, getFeature(t[i+n]))
        err = y-np.dot(p,getFeature(s))
        delta = err * (getFeature(s).reshape(27,1))
        p += ALPHA * delta.T


In [4]:
#plt.style.use('ggplot')

f, axarr = plt.subplots(8, 3, sharex='col', sharey='row')

for s in grid_states:
    if s in terminals:
        continue
    print s
    i = int(s)/3
    j = int(s)%3
    prob = np.dot(p,getFeature(s))
    prob /= np.sum(prob)
    v=[]
    v.append(np.dot(prob,getFeature('Wall')))
    v.append(np.dot(prob,getFeature('Exit')))
    v.append(np.dot(prob,getFeature('Open Space')))
    axarr[i,j].bar([0,1,2], v, 0.5)
    print "Exit:", np.dot(prob,getFeature('Exit'))
    print "Wall:", np.dot(prob,getFeature('Wall'))
    print "Open Space:", np.dot(prob,getFeature('Open Space'))
    print "\n"

00
Exit: 0.853669895937
Wall: 0.146330104063
Open Space: 0.0


01
Exit: 1.0
Wall: 0.0
Open Space: 0.0


02
Exit: 0.838469105135
Wall: 0.0
Open Space: 0.161530894865


03
Exit: 0.731688681524
Wall: 0.268311318476
Open Space: 0.0


04
Exit: 0.916501340029
Wall: 0.0280829454259
Open Space: 0.055415714545


05
Exit: 0.824004535991
Wall: 0.0
Open Space: 0.175995464009


06
Exit: 0.694277124474
Wall: 0.283396931026
Open Space: 0.0223259445005


07
Exit: 0.845910655024
Wall: 0.0799434527236
Open Space: 0.0741458922519


08
Exit: 0.709311924529
Wall: 0.0129906839566
Open Space: 0.277697391514


09
Exit: 0.624036660591
Wall: 0.351181431033
Open Space: 0.0247819083761


10
Exit: 0.78113134704
Wall: 0.102938660684
Open Space: 0.115929992277


11
Exit: 0.601582590246
Wall: 0.0575399441314
Open Space: 0.340877465623


12
Exit: 0.52930506348
Wall: 0.42055257853
Open Space: 0.0501423579896


13
Exit: 0.686964811041
Wall: 0.139953945307
Open Space: 0.173081243653


14
Exit: 0.567690759252
Wall: 0.0769

In [5]:
plt.show()